In [ ]:
import random

In [16]:

%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [14]:
def string_to_genes(string):
    return [int(bit) for bit in string]

def genes_to_string(genes):
    return ''.join(map(str, genes))

class Individual:
    def __init__(self, genes=None, gene_length=10, fitness_function=None):
        self.gene_length = gene_length
        self.genes = genes if genes else self.create_genes()
        self.fitness_function = fitness_function

    def create_genes(self):
        return [random.randint(0, 1) for _ in range(self.gene_length)]

    def fitness(self):
        return self.fitness_function(self.genes)

    def mutate(self, mutation_points):
        for mutation_point in mutation_points:
            self.genes[mutation_point] = 1 - self.genes[mutation_point]

    def crossover(self, other, crossover_point):
        child1_genes = self.genes[:crossover_point] + other.genes[crossover_point:]
        child2_genes = other.genes[:crossover_point] + self.genes[crossover_point:]
        return Individual(child1_genes, fitness_function=self.fitness_function), Individual(child2_genes, fitness_function=self.fitness_function)

class GeneticAlgorithm:
    def __init__(self, population, num_generations, mutation_rate, num_parents, fitness_function):
        self.population = population
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.num_parents = num_parents

    def selection(self):
        return sorted(self.population, key=lambda individual: individual.fitness(), reverse=True)[:self.num_parents]

    def print_selection(self, parents):
        print("  Selection:")
        for parent in parents:
            print(f"    {genes_to_string(parent.genes)} - Fitness: {parent.fitness()}")

    def print_mutation(self, child, mutation_points):
        mutated_string = ''
        for i in range(len(child.genes)):
            if i in mutation_points:
                mutated_string += f"({child.genes[i]})"
            else:
                mutated_string += str(child.genes[i])

        print(f"  Mutation: {mutated_string} - Fitness: {child.fitness()}")

    def print_crossover(self, child1, child2, crossover_point):
        print(f"  Crossover:")
        print(f"    {genes_to_string(child1.genes[:crossover_point])}|{genes_to_string(child1.genes[crossover_point:])} - Fitness: {child1.fitness()}")
        print(f"    {genes_to_string(child2.genes[:crossover_point])}|{genes_to_string(child2.genes[crossover_point:])} - Fitness: {child2.fitness()}")

    def run(self):
        for generation in range(self.num_generations):
            print(f"\nGeneration {generation}:")

            parents = self.selection()
            self.print_selection(parents)

            next_generation = []

            for _ in range(len(self.population) // 2):
                parent1, parent2 = random.sample(parents, 2)
                crossover_point = random.randint(1, len(parent1.genes) - 1)
                child1, child2 = parent1.crossover(parent2, crossover_point)
                self.print_crossover(child1, child2, crossover_point)

                mutation_points = [i for i in range(len(child1.genes)) if random.random() < self.mutation_rate]
                child1.mutate(mutation_points)
                self.print_mutation(child1, mutation_points)

                mutation_points = [i for i in range(len(child2.genes)) if random.random() < self.mutation_rate]
                child2.mutate(mutation_points)
                self.print_mutation(child2, mutation_points)
                next_generation.extend([child1, child2])

            self.population = next_generation

        print(f"\nFinal Generation {self.num_generations}:")
        return max(self.population, key=lambda x: x.fitness())


In [15]:
def count_ones_fitness_function(genes):
    return sum(genes)

initial_population = [
        "1101001010",
        "0110100101",
        "0011001100",
        "1010101010",
        "1110001110",
        "0001110001",
        "1001100110",
        "0101010101"
        ]
gene_length = len(initial_population[0])
num_generations = 5
mutation_rate = 0.01
num_parents = 4

population = [Individual(string_to_genes(individual), gene_length, count_ones_fitness_function) for individual in initial_population]

ga = GeneticAlgorithm(population, num_generations, mutation_rate, num_parents, count_ones_fitness_function)
best_solution = ga.run()
print(f"Best solution: {genes_to_string(best_solution.genes)} Fitness: {best_solution.fitness()}")


Generation 0:
  Selection:
    1110001110 - Fitness: 6
    1101001010 - Fitness: 5
    0110100101 - Fitness: 5
    1010101010 - Fitness: 5
  Crossover:
    111|0100101 - Fitness: 6
    011|0001110 - Fitness: 5
  Mutation: 1110100101 - Fitness: 6
  Mutation: 0110001110 - Fitness: 5
  Crossover:
    110|0101010 - Fitness: 5
    101|1001010 - Fitness: 5
  Mutation: 1100101010 - Fitness: 5
  Mutation: 1011001010 - Fitness: 5
  Crossover:
    01|01001010 - Fitness: 4
    11|10100101 - Fitness: 6
  Mutation: 0101001010 - Fitness: 4
  Mutation: 1110100101 - Fitness: 6
  Crossover:
    011010|1110 - Fitness: 6
    111000|0101 - Fitness: 5
  Mutation: 011010(0)110 - Fitness: 5
  Mutation: 1110000101 - Fitness: 5

Generation 1:
  Selection:
    1110100101 - Fitness: 6
    1110100101 - Fitness: 6
    0110001110 - Fitness: 5
    1100101010 - Fitness: 5
  Crossover:
    1110100|101 - Fitness: 6
    1110100|101 - Fitness: 6
  Mutation: 1110100101 - Fitness: 6
  Mutation: 1110100101 - Fitness: 6
  C